In [1]:
#~10 Minutes to run
# import pysam
import shutil
import pandas as pd
import dask.dataframe as dd

In [2]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.expand_frame_repr', False)  
pd.set_option('display.max_columns', None)

# Data files:
Contains the main file and the files to be merged with it and other files that will be kept separately and merged whenever necessary.

In [3]:
#Main file
variant_summary_df = pd.read_csv("../input/capstone-data/all data for capstone/variant_summary.tsv", sep="\t", low_memory=False)
#Merged with Main file
allele_gene_df = pd.read_csv("../input/capstone-data/all data for capstone/allele_gene.tsv", sep="\t", low_memory=False)
gene_specific_summary_df = pd.read_csv("../input/capstone-data/all data for capstone/gene_specific_summary.tsv", sep="\t", low_memory=False,skiprows=1)
gene_condition_source_id_df = pd.read_csv("../input/capstone-data/all data for capstone/gene_condition_source_id.tsv", sep="\t", low_memory=False)

In [4]:
#Kept as separate files and merge whenever needed
hgvs4variation_dd = dd.read_csv("../input/capstone-data/all data for capstone/hgvs4variation.tsv", sep='\t', skiprows=15)
organization_summary_df = pd.read_csv("../input/capstone-data/all data for capstone/organization_summary.tsv", sep="\t", low_memory=False)
submission_summary_df = pd.read_csv("../input/capstone-data/all data for capstone/submission_summary.tsv", sep='\t', skiprows=18) 
summary_of_conflicting_interpretations_df = pd.read_csv("../input/capstone-data/all data for capstone/summary_of_conflicting_interpretations.tsv", sep="\t", low_memory=False)
var_citations_df = pd.read_csv("../input/capstone-data/all data for capstone/var_citations.tsv", sep="\t", low_memory=False)
cross_references_df = pd.read_csv("../input/capstone-data/all data for capstone/cross_references.tsv", sep="\t", low_memory=False)

# Processing vcf file

In [ ]:
with open("/kaggle/input/capstone-data/all data for capstone/clinvar (1).vcf", 'r') as file:
    vcf_data = [line for line in file if not line.startswith('##')]  
    
header = vcf_data[0].strip().split('\t')  

data = pd.DataFrame([line.strip().split('\t') for line in vcf_data[1:]], columns=header)
data.drop(columns = ['#CHROM','POS','ID','REF','ALT','QUAL','FILTER'], inplace = True)

retain = ['ALLELEID', 'MC', 'AF_EXAC', 'AF_ESP', 'AF_TGP']
def parse_info(info_str):
    items = [kv.split("=", 1) for kv in info_str.split(";") if "=" in kv]
    return {k: v for k, v in items if k in retain}

# Apply to Series and expand into DataFrame
final = data['INFO'].apply(parse_info).apply(pd.Series)

# Processing the main file
Processing variant_summary file

In [ ]:
vsdf = variant_summary_df[
    (variant_summary_df["Assembly"] == "GRCh38") & 
    (variant_summary_df["Type"] == "single nucleotide variant")&
    (variant_summary_df["OriginSimple"]=="germline")
]

In [ ]:
vsdf_droplist = ["OriginSimple","SCVsForAggregateGermlineClassification","Type", 
                 "Assembly","ReferenceAllele","AlternateAllele","Cytogenetic",
                 "Guidelines","SomaticClinicalImpact","SomaticClinicalImpactLastEvaluated",
                 "ReviewStatusClinicalImpact","Oncogenicity","OncogenicityLastEvaluated",
                 "ReviewStatusOncogenicity","SCVsForAggregateSomaticClinicalImpact",
                 "SCVsForAggregateOncogenicityClassification","nsv/esv (dbVar)","ClinSigSimple","SubmitterCategories"]
vsdf = vsdf.drop(columns=vsdf_droplist)

## Merging allele_gene_df

added : 
- Name : full name of the gene
- GenesPerAlleleID : number of genes related to the allele
- Category : type of allele-gene relationship

In [ ]:
vsdf1 = vsdf.merge(
    allele_gene_df[["#AlleleID", "GeneID", "Name", "GenesPerAlleleID", "Category"]],
    on=["#AlleleID", "GeneID"],
    how="left"
)
vsdf1.rename(columns={"Name_x": "HGVS_Notation","Name_y": "GeneName"}, inplace=True)
vsdf1 = vsdf1.dropna()
# done adding allele_gene_df

## 2 main files:
- Shifted towards 2 main dataframes 1 for allele information and the other for gene info
- Can be merged when needed
    - merge on `GeneID` 

In [ ]:
gene_df = vsdf1[["GeneID", "GeneSymbol", "GeneName"]].drop_duplicates(subset="GeneID")
allele_df = vsdf1.drop(columns=["GeneSymbol", "GeneName"])

In [ ]:
allele_df.rename(columns = {"#AlleleID": "AlleleID"},inplace = True)

## Merging gene_condition_source_id to gene_df 
- adding a new column called `GeneLevelDisease` to gene_df

In [ ]:
# Group multiple disease names per GeneID into a single string
gene_disease_grouped = (
    gene_condition_source_id_df
    .groupby("#GeneID")["DiseaseName"]
    .apply(lambda x: "; ".join(sorted(set(x))))
    .reset_index()
)

# Merge with vsdf2
gene_df = gene_df.merge(gene_disease_grouped, left_on="GeneID",right_on="#GeneID", how="left")

# Rename and fill nulls
gene_df.rename(columns={"DiseaseName": "GeneLevelDisease"}, inplace=True)
gene_df["GeneLevelDisease"].fillna("No gene-disease association has been submitted to ClinVar for this gene.", inplace=True)

In [ ]:
gene_df = gene_df.merge(
    gene_specific_summary_df[
        ['GeneID', 'Total_submissions', 'Total_alleles', 'Submissions_reporting_this_gene',
         'Alleles_reported_Pathogenic_Likely_pathogenic', 'Number_uncertain', 'Number_with_conflicts']
    ],
    on='GeneID',
    how='left'
)

In [ ]:
gene_df["Submissions_reporting_this_gene"] = pd.to_numeric(gene_df["Submissions_reporting_this_gene"], errors="coerce")
gene_df["Alleles_reported_Pathogenic_Likely_pathogenic"] = pd.to_numeric(gene_df["Alleles_reported_Pathogenic_Likely_pathogenic"], errors="coerce")
gene_df["Number_uncertain"] = pd.to_numeric(gene_df["Number_uncertain"], errors="coerce")
gene_df["Number_with_conflicts"] = pd.to_numeric(gene_df["Number_with_conflicts"], errors="coerce")

Number of nulls: 
```
GeneID                                               0 
GeneSymbol                                           0
GeneName                                             0
#GeneID                                          13799 -> dropped
GeneLevelDisease                                     0
Total_submissions                                    1
Total_alleles                                        1
Submissions_reporting_this_gene                     70
Alleles_reported_Pathogenic_Likely_pathogenic    10531
Number_uncertain                                    60
Number_with_conflicts                               60
```

- Filling numeric col with placeholder -1

In [ ]:
gene_df.drop(columns = "#GeneID",inplace = True)
gene_df.fillna(-1.0, inplace=True)
# placeholder for floating point missing values is -1

In [ ]:
gene_df.rename(columns={
    'Total_submissions': 'TotalSubmissions',
    'Total_alleles': 'TotalAlleles',
    'Submissions_reporting_this_gene': 'SubmissionsReportingThisGene',
    'Alleles_reported_Pathogenic_Likely_pathogenic': 'AllelesReportedPathogenicLikelyPathogenic',
    'Number_uncertain': 'NumberUncertain',
    'Number_with_conflicts': 'NumberWithConflicts'
}, inplace=True)

allele_df.rename(columns={
    'HGVS_Notation': 'HgvsNotation',
    'HGNC_ID':'HGNCID'
}, inplace=True)


In [ ]:
allele_df = allele_df.merge(AF_MC_df[['AlleleID','MC']],on = "AlleleID",how = 'left')
allele_df = allele_df[~allele_df.duplicated(keep='first')].copy

In [17]:
del variant_summary_df,allele_gene_df,gene_specific_summary_df,gene_condition_source_id_df,vsdf

In [ ]:
final.rename(columns = {'ALLELEID':'AlleleID'},inplace = True)
allele_df['AlleleID'] = allele_df['AlleleID'].astype(int)
final['AlleleID'] = final['AlleleID'].astype(int)

allowed_ids = set(allele_df['AlleleID'])

final_fil = final[final['AlleleID'].isin(allowed_ids)]
# final_fil.drop(columns =['rs'],inplace = True)

allele_df = allele_df.merge(
    final_fil,
    on='AlleleID',
    how='left'
)

final_fil["variant_key"] = (
    final_fil["Chromosome"].astype(str) + "-" +
    final_fil["Start"].astype(str) + "-" +
    final_fil["ReferenceAlleleVCF"] + "-" +
    final_fil["AlternateAlleleVCF"]
)
final_fil.drop(columns = ['Chromosome','ReferenceAlleleVCF','AlternateAlleleVCF','Start'],inplace = True)

In [19]:
# Separate file for hgvs4variation 
hgvs4variation_dd = hgvs4variation_dd[hgvs4variation_dd.Assembly == 'GRCh38']
hgvs4variation_dd = hgvs4variation_dd.drop(columns = "Assembly")
hgvs4variation_df = hgvs4variation_dd.compute()
hgvs4variation_df = hgvs4variation_df[hgvs4variation_df['Type'] != 'genomic, Haplotype']
hgvs4variation_df.drop(columns = ['UsedForNaming','Submitted','OnRefSeqGene','#Symbol','GeneID','Type','ProteinExpression','ProteinChange'],inplace = True)

allowed_pairs = set(allele_df[['AlleleID', 'VariationID']].apply(tuple, axis=1))

filtered_hgvs = hgvs4variation_df[
    hgvs4variation_df[['AlleleID', 'VariationID']].apply(tuple, axis=1).isin(allowed_pairs)
]
# Keys to merge on VariationID , AlleleID

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:199: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [20]:
var_citations_df.rename(columns = {"#AlleleID":"AlleleID"},inplace = True)

var_citations_df.drop(columns = ["nsv"],inplace = True)
var_citations_df.fillna("-1",inplace = True) # rs , org_id

allowed_pairs = set(allele_df[['AlleleID', 'VariationID']].apply(tuple, axis=1))

filtered_var_citations = var_citations_df[
    var_citations_df[['AlleleID', 'VariationID']].apply(tuple, axis=1).isin(allowed_pairs)
]
# used to link alleleid|variationid to citation and also organization

/tmp/ipykernel_36/455290775.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  var_citations_df.fillna("-1",inplace = True) # rs , org_id


In [21]:
# separate file for organization_summary_df
organization_summary_df.drop(
    columns = ['street address', 'city', 'country',
               'date last submitted','novel and updates','somatic clinical impact values submitted',
               'somatic oncogenicity values submitted'],inplace = True)

organization_summary_df["clinical significance categories submitted"] = organization_summary_df["clinical significance categories submitted"].fillna("Not specified")

organization_summary_df.rename(columns={
    '#organization': 'OrganizationName',
    'organization ID': 'OrganizationID',
    'institution type': 'InstitutionType',
    'number of ClinVar submissions': 'NumberOfClinVarSubmissions',
    'maximum review status': 'MaximumReviewStatus',
    'collection methods': 'CollectionMethods',
    'clinical significance categories submitted': 'ClinicalSignificanceCategoriesSubmitted',
    'number of submissions from clinical testing': 'NumberOfSubmissionsFromClinicalTesting',
    'number of submissions from research': 'NumberOfSubmissionsFromResearch',
    'number of submissions from literature only': 'NumberOfSubmissionsFromLiteratureOnly',
    'number of submissions from curation': 'NumberOfSubmissionsFromCuration',
    'number of submissions from phenotyping': 'NumberOfSubmissionsFromPhenotyping'
}, inplace=True)

# Need to use var_citations_df because that has allele|variation to org_id

In [22]:
# separate file for submission_summary_df
submission_summary_df.rename(columns = {"#VariationID":"VariationID"},inplace = True)

submission_summary_df.drop(columns = ['SomaticClinicalImpact','Oncogenicity','DateLastEvaluated','SCV'],
                          inplace = True)
submission_summary_df["Description"] = submission_summary_df["Description"].fillna("Not Provided")


allowed_ids = set(allele_df['VariationID'])

filtered_submission_summary = submission_summary_df[
    submission_summary_df['VariationID'].isin(allowed_ids)
]
# Keys to merge on VariationID

In [ ]:
# separate file for conflict_submission_summary_df

summary_of_conflicting_interpretations_df.drop(columns = ['#Gene_Symbol','Submitter1_SCV',
                                      'Submitter2_SCV','Submitter1_LastEval','Submitter2_LastEval','Submitter1_Sub_Condition','Submitter2_Sub_Condition'],
                          inplace = True)

summary_of_conflicting_interpretations_df.rename(columns={"NCBI_Variation_ID": "VariationID"}, inplace=True)

summary_of_conflicting_interpretations_df["Submitter1_Description"] = summary_of_conflicting_interpretations_df["Submitter1_Description"].fillna("Not provided")
summary_of_conflicting_interpretations_df["Submitter2_Description"] = summary_of_conflicting_interpretations_df["Submitter2_Description"].fillna("Not provided")

allowed_ids = set(allele_df['VariationID'])

filtered_summary_of_conflicting_interpretations = summary_of_conflicting_interpretations_df[
    summary_of_conflicting_interpretations_df['VariationID'].isin(allowed_ids)
]
# Keys to merge on VariationID

In [24]:
allowed_ids = set(allele_df['AlleleID'])

filtered_cross_references = cross_references_df[
    cross_references_df['#AlleleID'].isin(allowed_ids)
]

In [25]:
final_fil.drop(columns = ['AF_EXAC','AF_TGP','AF_ESP'],inplace = True)

In [38]:
gene_df.to_csv("gene_df.tsv",sep = "\t",index=False )
allele_df.to_csv("allele_df.tsv",sep = "\t",index=False )
filtered_hgvs.to_csv("hgvs4variation_df.tsv",sep = "\t",index=False)
organization_summary_df.to_csv("organization_summary_df_1.tsv",sep = "\t",index=False )
filtered_submission_summary.to_csv("submission_summary_df.tsv",sep = "\t",index=False )
filtered_summary_of_conflicting_interpretations.to_csv("summary_of_conflicting_interpretations_df.tsv",sep = "\t",index=False )
filtered_var_citations.to_csv("var_citations_df.tsv",sep = "\t",index=False )
filtered_cross_references.to_csv("cross_references_df.tsv",sep = "\t",index=False )
final_fil.to_csv("Allelefreq_MC.tsv",sep = "\t")
# for external dbs

In [39]:
import os
import zipfile
from tqdm import tqdm

source_dir = "/kaggle/working"
output_zip = "/kaggle/working/final_dataset_all.zip"

# Collect all file paths
file_paths = []
for root, _, files in os.walk(source_dir):
    for file in files:
        full_path = os.path.join(root, file)
        # Exclude the output zip itself if rerunning
        if full_path != output_zip:
            file_paths.append(full_path)

# Create zip with progress bar
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in tqdm(file_paths, desc="Zipping files"):
        arcname = os.path.relpath(file, start=source_dir)
        zipf.write(file, arcname)


Zipping files: 100%|██████████| 9/9 [02:19<00:00, 15.55s/it]


# Create SQL duckdb file

In [ ]:
base_path = "/kaggle/input/preprocessed-capstone-data-3/data"

gene_df = pd.read_csv(f"{base_path}/gene_df.tsv", sep="\t")
allele_df = pd.read_csv(f"{base_path}/allele_df.tsv", sep="\t")
hgvs4variation_df = pd.read_csv(f"{base_path}/hgvs4variation_df.tsv", sep="\t")
cross_references_df = pd.read_csv(f"{base_path}/cross_references_df.tsv", sep="\t")
organization_summary_df = pd.read_csv(f"{base_path}/organization_summary_df_1.tsv", sep="\t")
submission_summary_df = pd.read_csv(f"{base_path}/submission_summary_df.tsv", sep="\t")
summary_of_conflicting_interpretations_df = pd.read_csv(f"{base_path}/summary_of_conflicting_interpretations_df.tsv", sep="\t")
var_citations_df = pd.read_csv(f"{base_path}/var_citations_df.tsv", sep="\t")

In [ ]:
with duckdb.connect('Capstone_data.duckdb') as con:
    con.execute("CREATE TABLE gene AS SELECT * FROM gene_df")
    con.execute("CREATE TABLE allele AS SELECT * FROM allele_df")
    con.execute("CREATE TABLE submission_summary AS SELECT * FROM submission_summary_df")
    con.execute("CREATE TABLE summary_of_conflicting_interpretations AS SELECT * FROM summary_of_conflicting_interpretations_df")
    con.execute("CREATE TABLE organization_summary AS SELECT * FROM organization_summary_df")
    con.execute("CREATE TABLE var_citations AS SELECT * FROM var_citations_df")
    con.execute("CREATE TABLE cross_references AS SELECT * FROM cross_references_df")
    con.execute("CREATE TABLE hgvs4variation AS SELECT * FROM hgvs4variation_df")

# Normalising some column names : 

In [ ]:
import duckdb,shutil,os

In [ ]:
shutil.copy("/kaggle/input/capstone-sql-db/Capstone_data (1).duckdb", "/kaggle/working/Capstone_data_1.duckdb")

# Connect to the copied file
con = duckdb.connect("/kaggle/working/Capstone_data_1.duckdb")

In [ ]:
result = con.execute("SHOW TABLES;").fetchall()

for table in result:
    table_name = table[0]
    print(f"\nTable: {table_name}")
    columns = con.execute(f"DESCRIBE {table_name}").fetchall()
    for col in columns:
        print(col[0])


In [ ]:
queries = [
    # Drop Unnamed: 0 from allele
    'ALTER TABLE allele DROP COLUMN "Unnamed: 0";',

    # allele columns
    'ALTER TABLE allele RENAME COLUMN "RCVaccession" TO RCVAccession;',

    # cross_references columns
    'ALTER TABLE cross_references RENAME COLUMN "#AlleleID" TO AlleleID;',
    'ALTER TABLE cross_references RENAME COLUMN "last_updated" TO LastUpdated;',



    # summary_of_conflicting_interpretations columns
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "ClinVar_Preferred" TO ClinvarPreferred;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter1_ClinSig" TO Submitter1Clinsig;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter1_ReviewStatus" TO Submitter1Reviewstatus;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter1_Description" TO Submitter1Description;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter2_ClinSig" TO Submitter2Clinsig;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter2_ReviewStatus" TO Submitter2Reviewstatus;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter2_Description" TO Submitter2Description;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Rank_diff" TO RankDiff;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Conflict_Reported" TO ConflictReported;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Variant_type" TO VariantType;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter1_Method" TO Submitter1Method;',
    'ALTER TABLE summary_of_conflicting_interpretations RENAME COLUMN "Submitter2_Method" TO Submitter2Method;',

    # var_citations columns
    'ALTER TABLE var_citations DROP COLUMN "rs"',
    'ALTER TABLE var_citations RENAME COLUMN "citation_source" TO CitationSource;',
    'ALTER TABLE var_citations RENAME COLUMN "citation_id" TO CitationID;',
    'ALTER TABLE var_citations RENAME COLUMN "organization_ids" TO OrganizationID;',
]

for q in queries:
    con.execute(q)


In [ ]:
con.close()